# Projekt B: Supply Chain Data Cleaning Pipeline 🚛🧹
**Autor:** Kilian Sender
**Status:** Work in Progress


## 1. Setup & Datenimport


### Datenquelle
Der Datensatz **"Supply Chain Shipment Pricing Data"** stammt von **Pushpit Kamboj**.
* **Original-Link:** https://www.kaggle.com/datasets/pushpitkamboj/logistics-data-containing-real-world-data?resource=download
* **Lizenz:** CC0: Public Domain

Hier wird die CSV inklusive der nötigen Bibliotheken geladen

In [1]:
import pandas as pd

# Versuch 1: Direkt im aktuellen Ordner laden
try:
    df_rohdaten = pd.read_csv('incom2024_delay_example_dataset.csv')
    print("✅ Datei erfolgreich geladen!")
except FileNotFoundError:
    print("❌ Datei immer noch nicht gefunden. Prüfe den Pfad!")
    
    # Detektiv-Hilfe: Zeig mir, wo ich bin und was hier liegt
    import os
    print(f"\nIch bin hier: {os.getcwd()}")
    print("Hier liegen folgende Dateien:", os.listdir())

✅ Datei erfolgreich geladen!


## 2. Explorative Analyse (Der Detektiv-Blick)
Wir verschaffen uns einen Überblick über Datentypen, fehlende Werte und offensichtliche Fehler.

In [2]:
df_rohdaten.head()

,payment_type,profit_per_order,sales_per_customer,category_id,category_name,customer_city,customer_country,customer_id,customer_segment,customer_state,...,order_region,order_state,order_status,product_card_id,product_category_id,product_name,product_price,shipping_date,shipping_mode,label
0,DEBIT,34.448338,92.49099,9.0,Cardio Equipment,Caguas,Puerto Rico,12097.6830,Consumer,PR,...,Western Europe,Vienna,COMPLETE,191.0,9.0,Nike Men's Free 5.0+ Running Shoe,99.99,2015-08-13 00:00:00+01:00,Standard Class,-1
1,TRANSFER,91.193540,181.99008,48.0,Water Sports,Albuquerque,EE. UU.,5108.1045,Consumer,CA,...,South America,Buenos Aires,PENDING,1073.0,48.0,Pelican Sunstream 100 Kayak,199.99,2017-04-09 00:00:00+01:00,Standard Class,-1
2,DEBIT,8.313806,89.96643,46.0,Indoor/Outdoor Games,Amarillo,Puerto Rico,4293.4478,Consumer,PR,...,Western Europe,Nord-Pas-de-Calais-Picardy,COMPLETE,1014.0,46.0,O'Brien Men's Neoprene Life Vest,49.98,2015-03-18 00:00:00+00:00,Second Class,1
3,TRANSFER,-89.463196,99.15065,17.0,Cleats,Caguas,Puerto Rico,546.5306,Consumer,PR,...,Central America,Santa Ana,PROCESSING,365.0,17.0,Perfect Fitness Perfect Rip Deck,59.99,2017-03-18 00:00:00+00:00,Second Class,0
4,DEBIT,44.722590,170.97824,48.0,Water Sports,Peabody,EE. UU.,1546.3980,Consumer,CA,...,Central America,Illinois,COMPLETE,1073.0,48.0,Pelican Sunstream 100 Kayak,199.99,2015-03-30 00:00:00+01:00,Standard Class,1


## 3. Datenbereinigung (Technical Cleaning)
Hier standardisieren wir Datentypen und Formate, um technische Konsistenz herzustellen.

**Durchgeführte Schritte:**
1.  **Zip-Codes:** Entfernen von Dezimalstellen und Padding auf 5 Ziffern (String-Format).
2.  **Finanzdaten:** Runden aller Währungsspalten (z.B. `product_price`, `sales`) auf 2 Nachkommastellen.
3.  **IDs & Mengen:** Umwandlung von Floats in Integer (Entfernung von synthetischem Rauschen).
4.  **Datumsformate:** Konvertierung von String zu `datetime64` (UTC), um Zeitberechnungen zu ermöglichen.

In [3]:
df = df_rohdaten.copy()


# Wir wandeln erst in 'int' (schneidet Komma ab), dann in 'str' weil sonst keine '0' vorne stehen kann für z.B. US-Zipcodes
df['Zip_Clean'] = pd.to_numeric(df['customer_zipcode'], errors='coerce').fillna(0).astype(int).astype(str)

# Alles unter 6 Ziffern wird vorne mit '0' aufgefüllt
df['Zip_Clean'] = df['Zip_Clean'].str.zfill(5)

# --- FIX: GELD-SPALTEN (RUNDEN) ---
# Preise können technisch nicht mehr als 2 Nachkommastellen haben.
# Wir beheben das "Rauschen" (z.B. 47.9397 -> 47.94).

money_cols = [
    'product_price',
    'sales',
    'order_item_total_amount',
    'profit_per_order',
    'sales_per_customer',
    'order_profit_per_order',
    'order_item_discount',
    'order_item_product_price',

    ]

# Profi-Tipp: Wir runden alle Geld-Spalten auf einmal
for col in money_cols:
    # Checken, ob Spalte existiert (falls wir uns vertippt haben)
    if col in df.columns:
        df[col] = df[col].round(2)

# Check
print("Preise nach Bereinigung:")
print(df[money_cols].head())

# --- FIX: PROZENTWERTE (RATES & RATIOS) ---
# Auch hier runden wir auf 2 Stellen (0.1234 -> 0.12)
rate_cols = ['order_item_discount_rate', 'order_item_profit_ratio']

for col in rate_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').round(2)

# --- FIX: INTEGERS (IDs & Stückzahlen) ---
# Diese Spalten dürfen keine Kommastellen haben. Wir machen radikal sauber.

# Liste der Spalten, die DEFINITIV Integer sein sollen
cols_to_int = [
    'order_item_quantity',
    'order_item_id',
    'order_item_cardprod_id',
    'order_id',
    'order_customer_id',
    'department_id', 
    'category_id',
    'department_id',
    'customer_id'
    ]

for col in cols_to_int:
    # 1. Erst in Zahl wandeln (um Fehler zu fangen)
    # 2. NaNs mit 0 füllen (damit astype(int) nicht crasht)
    # 3. Hart in Integer wandeln
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)


# --- CHECK ---
print("Neue Datentypen:")
print(df[cols_to_int].dtypes)
print("\nBeispiel-Daten:")
print(df[cols_to_int].head(3))

# --- FIX: TEXT (STRINGS) ---
# Entfernt Leerzeichen am Anfang/Ende von ALLEN Text-Spalten
# select_dtypes(include='object') greift sich automatisch alle Text-Spalten
df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())


# --- FIX: PRODUKT-IDS ---
# Auch hier gibt es "Noise" (Kommazahlen), die wir abschneiden müssen.
cols_to_fix = ['product_card_id', 'product_category_id']

for col in cols_to_fix:
    # 1. Fillna(0) falls NaNs da sind (Sicherheitsnetz)
    # 2. astype(int) schneidet das Komma ab
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# --- FIX: DATUMSFORMATE ---

# Konvertierung zu DateTime
# pandas versucht automatisch, das Format zu erraten.
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce', utc=True)
df['shipping_date'] = pd.to_datetime(df['shipping_date'], errors='coerce',utc=True)

# Check hat alles geklappt?
df[['Zip_Clean','order_date', 'shipping_date','product_card_id', 'product_category_id']].head()
print("Cleaning durchgelaufen.")


Preise nach Bereinigung:
   product_price   sales  order_item_total_amount  profit_per_order  \
0          99.99   99.99                    84.99             34.45   
1         199.99  199.99                   181.99             91.19   
2          49.98   99.96                    93.81              8.31   
3          59.99  119.98                    99.89            -89.46   
4         199.99  199.99                   171.08             44.72   

   sales_per_customer  order_profit_per_order  order_item_discount  \
0               92.49                   32.08                12.62   
1              181.99                   91.24                16.50   
2               89.97                    6.97                 6.60   
3               99.15                  -95.40                16.94   
4              170.98                   44.57                29.99   

   order_item_product_price  
0                     99.99  
1                    199.99  
2                     49.98  
3      

In [4]:
# Kurzer Blick: Haben wir Mist gebaut?
df[['customer_zipcode', 'Zip_Clean']].head()

,customer_zipcode,Zip_Clean
0,725.0000,00725
1,92745.1600,92745
2,2457.7297,02457
3,725.0000,00725
4,95118.6000,95118


Entscheidung: Die Spalte customer_zipcode enthält synthetisches Rauschen (Dezimalstellen) und Inkonsistenzen. Sie wird für die Analyse ignoriert. Stattdessen nutzen wir order_country für die geografische Aggregation.

## 4. Logik-Prüfung & Feature Engineering
Wir berechnen die tatsächliche Lieferzeit (`actual_shipping_days`) und prüfen die Daten auf physikalische Plausibilität.

### ⚠️ Anomalie-Erkennung: "Zeitreisende Pakete"
Die Analyse der Lieferzeiten zeigt Datensätze mit **negativen Werten** (Versanddatum liegt *vor* dem Bestelldatum).
* **Befund:** Ca. 40% der Daten sind betroffen. Extremwert: -1.429 Tage.
* **Maßnahme:** Bereinigung (Löschen) dieser Zeilen, um die statistische Integrität für Lead-Time-Analysen zu gewährleisten.

In [5]:
# Berechnung: Versanddatum - Bestelldatum
df['actual_shipping_days'] = (df['shipping_date'] - df['order_date']).dt.days

# PLAUSIBILITÄTS-CHECK:
# Gibt es negative Lieferzeiten? (Versand VOR Bestellung?)
negative_days = df[df['actual_shipping_days'] < 0]

print("--- ANALYSE DER ZEITREISENDEN ---")
print(f"Anzahl unlogischer Datensätze (negativ): {len(negative_days)}")
print("\nBeispiel für Fehler:")
print(negative_days[['order_date', 'shipping_date', 'actual_shipping_days']].head(3))

# Visualisierung des Fehlers (Optional, aber cool)
print("\nStatistik der Fehler:")
print(negative_days['actual_shipping_days'].describe())

--- ANALYSE DER ZEITREISENDEN ---
Anzahl unlogischer Datensätze (negativ): 5941

Beispiel für Fehler:
                 order_date             shipping_date  actual_shipping_days
3 2017-05-30 23:00:00+00:00 2017-03-18 00:00:00+00:00                   -74
7 2016-06-08 23:00:00+00:00 2016-04-23 23:00:00+00:00                   -46
9 2017-08-28 23:00:00+00:00 2017-04-27 23:00:00+00:00                  -123

Statistik der Fehler:
count    5941.000000
mean      -72.394041
std        87.001059
min     -1429.000000
25%      -100.000000
50%       -40.000000
75%       -15.000000
max        -1.000000
Name: actual_shipping_days, dtype: float64


### ⚠️ Daten-Anomalie entdeckt!
Die Analyse zeigt 5.941 Datensätze mit **negativer Lieferzeit** (bis zu -1429 Tage). Das ist physikalisch unmöglich (Versand fast 4 Jahre vor Bestellung).
**Ursache:** Vermutlich Fehler bei der Datenerfassung oder Artefakte aus der synthetischen Generierung des Datensatzes.
**Entscheidung:** Da wir die Zeitreise noch nicht erfunden haben, werden diese Zeilen als "Datenfehler" behandelt und **entfernt**, um den Durchschnitt (Mean) nicht zu verfälschen.

In [6]:
# BEREINIGUNG
# Wir behalten nur Zeilen, bei denen die Lieferzeit >= 0 ist
rows_before = len(df)
df = df[df['actual_shipping_days'] >= 0]
rows_after = len(df)

print(f"Bereinigung abgeschlossen.")
print(f"Gelöschte Zeilen: {rows_before - rows_after}")
print(f"Verbleibende Datenbasis: {rows_after}")

Bereinigung abgeschlossen.
Gelöschte Zeilen: 5941
Verbleibende Datenbasis: 9608


Aufgrund der massiven Inkonsistenz (ca. 40% der Daten verletzen die Kausalität) entscheiden wir uns für Qualität statt Quantität und entfernen diese Datensätze, um nachgelagerte Analysen (Lead Time Prediction) nicht zu gefährden.

## 5. Finaler Hausputz (Cleanup)
Nachdem die neuen, sauberen Spalten (z.B. `Zip_Clean`) erfolgreich erstellt wurden, entfernen wir die veralteten Originalspalten und führen einen letzten Datentyp-Check durch.

**Ziel:** Ein schlanker, analysebereiter Datensatz ohne Redundanzen.

In [7]:
# --- FINALER HAUSPUTZ ---
# Jetzt, wo wir sicher sind, dass Zip_Clean funktioniert:
# Weg mit dem alten Müll!

df = df.drop(columns=['customer_zipcode'])
df = df.rename(columns={'Zip_Clean': 'customer_zipcode'})

print("✅ Alte Spalten gelöscht. Dataset ist jetzt sauber.")
print(f"Verbleibende Floats: {df.select_dtypes(include=['float']).columns.tolist()}")

✅ Alte Spalten gelöscht. Dataset ist jetzt sauber.
Verbleibende Floats: ['profit_per_order', 'sales_per_customer', 'latitude', 'longitude', 'order_item_discount', 'order_item_discount_rate', 'order_item_product_price', 'order_item_profit_ratio', 'sales', 'order_item_total_amount', 'order_profit_per_order', 'product_price']


## 6. Export
Der bereinigte Datensatz wird als CSV gespeichert, bereit für das Dashboarding (PowerBI/Tableau).

In [8]:
# Speichern ohne den Index (die Zeilennummern 0,1,2... brauchen wir nicht in der CSV)
output_filename = 'cleaned_supply_chain_data.csv'
df.to_csv(output_filename, index=False)

print(f"✅ Datei erfolgreich gespeichert als: {output_filename}")
print(f"Dateigröße: {df.shape[0]} Zeilen, {df.shape[1]} Spalten")

✅ Datei erfolgreich gespeichert als: cleaned_supply_chain_data.csv
Dateigröße: 9608 Zeilen, 42 Spalten
